In [ ]:
%reload_ext autoreload
%autoreload

from WebTools.emailcli import *
from getpass import getpass
from simplecrypt import encrypt, decrypt

server = 'imap.gmail.com'
user = input('User:')
password = getpass('Pass:')

mail = ImapClient(15)
mail.SetDebug(True)

mail.setmailserver(server)
mail.setUserPass(user, password)
mail.login()

mail.getMailBoxes()
print('-'*60)




In [ ]:
mail.SetDebug(False)

In [ ]:
#mailbox = '"[Gmail]/Sent Mail"'
mailbox = 'INBOX'
print(mail.getMailBoxStatus(mailbox))

mail.selectMailbox(mailbox)
msg_unseen = mail.getSelectedMailBoxUnseens_UID()
print('UnSeen :', msg_unseen)

msg_deleted = mail.getSelectedMailBoxDeleted_UID()
print('Marked Deleted', msg_deleted)


In [ ]:

print('Unseen:')
mail.PeekMailbox_Unseen(showFullHeader=False, peekbody=False)

print('All messages:')
print('#'*90)
mail.PeekMailbox_All(showFullHeader=False)


In [ ]:

ids = mail.SearchMail(All='APS URSI 2016')
print('number of emails found: %s\n'%(len(ids)))
print(ids)

### Read emails

In [ ]:
msg_id = b'17733'

flags = mail.readMailFlags(msg_id)
print('FLAGS: ', flags)
envelope = mail.readMailEnvelope(msg_id)
print('ENVELOPE: ', envelope)

header = mail.readMailHeader(msg_id)
#print(type(header))
print('HEADER: ', header)

text_list, html_list = mail.readMailBody(msg_id)

from IPython.core.display import display, DisplayObject, HTML

html_text = None

print('='*30, 'HTML', '='*30)
for html_part in html_list:
    #display(html_part)     ## displays html text
    display(HTML(html_part))
    print('-'*60)

print('='*30, 'TEXT', '='*30)
for text_part in text_list:
    print(text_part)
    print('-'*60)


In [ ]:
#mail.MarkAsUnread(b'3309')

#mail.MarkAsRead(b'3309')

#mail.DeleteMessage(b'3309')
#mail.ExpungeDeletedMessages()

In [ ]:
mail.IssueUIDCommand('fetch', b'3304', '(FLAGS ENVELOPE)')

### send emails

In [ ]:
## send email (SMTP)

from WebTools.emailcli import SMTPEmailer

smtphost = 'smtp.gmail.com'
smtpport = 587
From = input("User:")

To = ['nima.chamanara@polymtl.ca']
In_reply_To = None

Subject = 'Test'
Body = "Test\
\n\
Nima"

print('Message: \n', Body)


import os
cwd = os.getcwd()
Attachments = [os.path.join(cwd, 'attachments', 'test.txt')]

emailsender = SMTPEmailer(smtphost, smtpport, From=From, To=To, Subject=Subject, Body=Body, 
                          In_reply_To=In_reply_To, Attachments=Attachments)
emailsender.SetDebug(2)

res = emailsender.sendEMail(user, password)

if res:
    print('Email sent')
else:
    print('Problem sending email.')

### Raw data

In [ ]:
raw_mail = mail.getRawMail(b'17733')

In [ ]:
for i in range(len(raw_mail)):
    for j in range(len(raw_mail[i])):
        print('[%s][%s]'%(i, j))
        if type(raw_mail[i][j]) is not int:
            print(raw_mail[i][j].decode('latin-1'))
        else:
            print(raw_mail[i][j])
        print('-'*60)
    

In [ ]:
text = 'Voici les r=E9sum=E9s des pr=E9sentations qui auront lieu cet = apr=E8s-midi dans le cadre du cours ele6904. Bienvenu =E0 tous.'

import quopri
text_de = quopri.decodestring(text)

text_de.decode('iso-8859-1')

In [ ]:
import poplib
from email import parser

pop_conn = poplib.POP3_SSL('pop.gmail.com')
pop_conn.user(input("User:"))
pop_conn.pass_(getpass("Pass:"))
#Get messages from server:
#messages = [pop_conn.retr(i) for i in range(1, len(pop_conn.list()[1]) + 1)]
messages = [pop_conn.retr(i) for i in range(1, 10)]
# Concat message pieces:
messages = ["\n".join(mssg[1]) for mssg in messages]
#Parse message intom an email object:
messages = [parser.Parser().parsestr(mssg) for mssg in messages]
for message in messages:
    print(message['subject'])
pop_conn.quit()

In [ ]:
sys.stdout.encoding
